In [1]:
from scrapy import Selector
from selenium import webdriver
import requests
import urlparse
import os
import re

In [11]:
base_url = 'https://www.crunchbase.com/'
url = 'https://www.crunchbase.com/funding-rounds'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/600.7.12 (KHTML, like Gecko) Version/8.0.7 Safari/600.7.12'}

#response = requests.get(url, headers=headers)
browser = webdriver.Chrome()

In [12]:
with open('/tmp/sample.html', 'wb') as fout:
    fout.write(response.content)

In [6]:
# Company list
browser.get('file:///{}/sample.html'.format(os.getcwd()))
sel = Selector(text=browser.page_source)

In [12]:
# Company info
browser.get('file:///{}/raw_content.html'.format(os.getcwd()))
sel = Selector(text=browser.page_source)

In [9]:
# Extract funding dates from current page
funding_date = sel.xpath('//h2[@class="title_date"]')

# Extract companies urls for first funding date
date = funding_date[0].xpath('text()').extract()
company_list = [urlparse.urljoin(base_url, item) for item in funding_date[0].xpath('//div[@class="info-block"]/h4/a/@href').extract() if re.search('/organization/\S+', item)]

print(date)
print(company_list[0])

[u'July 27, 2016']
https://www.crunchbase.com/organization/shipt-2


In [55]:
#company_crunchbase_links = sel.xpath('//div[@class="info-block"]//a/@href').extract()
company_crunchbase_links = sel.xpath('//li["data-date"]//div[@class="info-block"]//a/@href').extract()
print(len(company_crunchbase_links))

company_crunchbase_link = urlparse.urljoin(base_url, company_crunchbase_links[0])
print(company_crunchbase_link)

company_crunchbase_link = urlparse.urljoin(base_url, company_crunchbase_links[-1])
print(company_crunchbase_link)

215
https://www.crunchbase.com/organization/shipt-2
https://www.crunchbase.com/funding-round/dd6299977f8af8fbfb1067172429e4b0/investors


In [39]:
company_response = requests.get('https://www.crunchbase.com/organization/shipt-2', headers=headers)

with open('/tmp/company_sample.html', 'wb') as fout:
    fout.write(company_response.content)

browser.get("file:///tmp/company_sample.html")
sel = Selector(text=browser.page_source)

In [34]:
company_name = company_sel.xpath('//h1[@id="profile_header_heading"]//a/text()').extract()
print(company_name)

company_description = company_sel.xpath('//div[@class="definition-list container"]//dd[2]/text()').extract()
print(company_description)

funding_round = company_sel.xpath('//table[@class="table container"]//a/text()').extract()[0]
print(funding_round)

funding_amount = company_sel.xpath('//table[@class="table container"]//td/text()').extract()[1]
print(funding_amount)

company_site_link = company_sel.xpath('//div[@class="definition-list container"]//dd[5]/a/@href').extract()
print(company_site_link)

company_linkedin_link = company_sel.xpath('//dd[@class="social-links"]//a[@class="icons linkedin"]/@href').extract()
print(company_linkedin_link)




[u'Shipt']
[u'Shipt is an on-demand grocery delivery service that gives customers direct access to thousands of grocery items through a mobile app.']
Series A
$20.1M / 
[u'https://www.shipt.com']
[u'https://www.linkedin.com/company/shipt?']


In [48]:
team_members = sel.xpath('//div[@class="base info-tab people"]//ul[@class="section-list container"]/li')
#print(team_members)
aa = team_members.xpath('//div[@class="info-block"]/div[@class="large"]//a[@class="follow_card"]/text()').extract()
bb = team_members.xpath('//div[@class="info-block"]/div[@class="large"]/h5/text()').extract()

print(bb, aa)
#team_member_crunchbase_link = urlparse.urljoin(base_url, team_members[0])


#team_member_response = requests.get(team_member_crunchbase_link, headers=headers)

#with open('/tmp/team_member_sample.html', 'wb') as fout:
#    fout.write(team_member_response.content)

#browser.get("file:///tmp/team_member_sample.html")
#team_member_sel = Selector(text=browser.page_source)

([u'CTO', u'Head of Business Development', u'Director of Digital Marketing', u'CEO', u'Mobile Developer', u'VP of Finance & Operations', u'CTO', u'Head of Business Development', u'Director of Digital Marketing', u'CEO', u'Mobile Developer', u'VP of Finance & Operations', u'CTO', u'Head of Business Development', u'Director of Digital Marketing', u'CEO', u'Mobile Developer', u'VP of Finance & Operations', u'CTO', u'Head of Business Development', u'Director of Digital Marketing', u'CEO', u'Mobile Developer', u'VP of Finance & Operations', u'CTO', u'Head of Business Development', u'Director of Digital Marketing', u'CEO', u'Mobile Developer', u'VP of Finance & Operations', u'CTO', u'Head of Business Development', u'Director of Digital Marketing', u'CEO', u'Mobile Developer', u'VP of Finance & Operations'], [u'Bryan Davis', u'Joe Manning', u'Missy Polhemus', u'Bill Smith', u'Patrick Sullivan', u'Jonathan Wilhite', u'Bryan Davis', u'Joe Manning', u'Missy Polhemus', u'Bill Smith', u'Patrick Su

In [45]:
team_member_full_name = team_member_sel.xpath('//h1[@id="profile_header_heading"]//a/text()').extract()
print(team_member_full_name)

team_member_position = team_member_sel.xpath('//div[@class="overview-stats"]//dd/text()').extract()[0]
print(team_member_position)

team_member_details = team_member_sel.xpath('').extract()
print(team_member_details)

team_member_linkedin_link = team_member_sel.xpath('').extract()
print(team_member_details)

team_member_crunchbase_link = team_member_sel.xpath('').extract()
print(team_member_crunchbase_link)

[u'Bryan Davis']
CTO 


ValueError: Invalid XPath: 